In [ ]:
import boto3
import json
import logging
from botocore.exceptions import ClientError
import zipfile
import os
import time


In [ ]:
# Instantiating clients
aws_lambda = boto3.client('lambda', region_name='us-east-1')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3', region_name='us-east-1')
sqs = boto3.client('sqs', region_name= "us-east-1")
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
role = iam_client.get_role(RoleName='LabRole')
role_arn = role['Role']['Arn']

In [ ]:
# --- Create S3 Bucket ---
def create_bucket(bucket_name, region='us-east-1'):
    """Create an S3 bucket in a specified region."""
    try:
        if region == 'us-east-1':
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
        print(f'Bucket {bucket_name} created successfully.')
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket_name = "final-summer99d"
create_bucket(bucket_name)

def clean_s3_bucket(bucket_name):
    """Delete all objects in the S3 bucket to start fresh."""
    try:
        objects = s3_client.list_objects_v2(Bucket=bucket_name).get('Contents', [])
        if objects:
            s3_client.delete_objects(
                Bucket=bucket_name,
                Delete={'Objects': [{'Key': obj['Key']} for obj in objects]}
            )
            print(f'Cleaned {len(objects)} existing objects from bucket {bucket_name}.')
        else:
            print(f'No existing objects in bucket {bucket_name}.')
    except ClientError as e:
        logging.error(e)
        print(f"Error cleaning S3 bucket: {str(e)}")

In [ ]:
# ----- Create a DynamoDB table ----
def create_dynamodb_table(table_name):
    """Create a DynamoDB table for survey responses."""
    try:
        dynamodb_client.create_table(
            TableName=table_name,
            KeySchema=[
                {'AttributeName': 'user_id', 'KeyType': 'HASH'},  # Partition key
                {'AttributeName': 'timestamp', 'KeyType': 'RANGE'}  # Sort key
            ],
            AttributeDefinitions=[
                {'AttributeName': 'user_id', 'AttributeType': 'S'},
                {'AttributeName': 'timestamp', 'AttributeType': 'S'}
            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        )
        dynamodb_client.get_waiter('table_exists').wait(TableName=table_name)
        print(f'DynamoDB table {table_name} created successfully.')
    except ClientError as e:
        logging.error(e)
        return False
    return True

# Set your beta table name
table_name = 'CyclicalBetaSurvey'

# Call the function to create the table
create_dynamodb_table(table_name)

In [ ]:
##running the lambda function for categorization.
# Step 4: Create Lambda Function 
with open('cat_lambda.zip', 'rb') as f:
    lambda_zip = f.read()
aws_lambda.create_function(
    FunctionName='cat_lambda',
    Runtime='python3.9',
    Role=role_arn,
    Handler='lambda_function.lambda_handler',
    Code=dict(ZipFile=lambda_zip),
    Timeout=30
)
print("Categorizing lambda function created.")

In [ ]:
# Step 3: Create SQS Queue
queue_url = sqs.create_queue(QueueName='beta_survey_queue_summer99d')['QueueUrl']
print(f"SQS queue created: {queue_url}")

In [ ]:
# Step 4: Get SQS Queue ARN
sqs_info = sqs.get_queue_attributes(QueueUrl=queue_url, AttributeNames=['QueueArn'])
sqs_arn = sqs_info['Attributes']['QueueArn']
print(f"SQS ARN: {sqs_arn}")

In [ ]:
# Step 5: Connect SQS to Lambda
aws_lambda.create_event_source_mapping(
    EventSourceArn=sqs_arn,
    FunctionName='cat_lambda',
    Enabled=True,
    BatchSize=10
)
print("SQS queue linked to Lambda function.")

print(f"SQS Queue URL: {queue_url}")

In [ ]:
def send_survey(survey_path, sqs_url):
    sqs = boto3.client('sqs', region_name='us-east-1')
    
    # Read the JSON survey file
    with open(survey_path, 'r') as f:
        survey_data = json.load(f)
    
    # Convert the survey data to a string
    survey_str = json.dumps(survey_data)
    
    # Send the survey data to the SQS queue
    response = sqs.send_message(
        QueueUrl=sqs_url,
        MessageBody=survey_str
    )
    
    # Check if the message was sent successfully
    if response.get('MessageId'):
        return 200
    return 400

In [ ]:
# List of survey files
survey_files = [
    'tests/test1.json',
    'tests/test2.json',
    'tests/test3.json',
    'tests/test4.json',
    'tests/test5.json',
    'tests/test6.json',
    'tests/test7.json',
    'tests/test8.json',
    'tests/test9.json',
    'tests/test10.json'
]

sqs_url = " "

for survey_file in survey_files:
    status = send_survey(survey_file, sqs_url)
    print(status)
    time.sleep(10)

In [ ]:
# --- Verify DynamoDB Records ---
table = dynamodb.Table(table_name)
dynamodb_records = table.scan()['Items']
print("\nDynamoDB records:")
if dynamodb_records:
    for record in dynamodb_records:
        print(record)
else:
    print("No records found in SurveyResponses_Summer99D table. Checking table status...")
    try:
        table_description = dynamodb_client.describe_table(TableName=table_name)
        print(f"Table status: {table_description['Table']['TableStatus']}")
        print(f"Table item count: {table_description['Table']['ItemCount']}")
    except ClientError as e:
        print(f"Error describing table: {str(e)}")

In [ ]:
# Delete each pipeline component if it still exists:
# Lambda
try:
    aws_lambda.delete_function(FunctionName='HW6')
    print("Lambda Function Deleted")
except aws_lambda.exceptions.ResourceNotFoundException:
    print("AWS Lambda Function Already Deleted")

# SQS
try:
    sqs.delete_queue(QueueUrl=sqs_url)
    print("SQS Queue Deleted")
except sqs.exceptions.QueueDoesNotExist:
    print("SQS Queue Already Deleted")

# DynamoDB
dynamodb = boto3.client('dynamodb', region_name= 'us-east-1')
try:
    response = dynamodb.delete_table(TableName='SurveyResponses_Summer99D_HW06')
    print("DynamoDB Table Deleted")
except dynamodb.exceptions.ResourceNotFoundException:
    print("DynamoDB Table Already Deleted")